## Class 8 Agenda:
  * **Feature Engineering: Standard Scaling/MinMaxScaling**
  * **Feature Engineering: Eliminating Skew/Normalizing Features**
  * **Feature Engineering: Handling Categorical Features**
  * **Feature Engineering: Handling Missing Values**
  * **Feature Engineering: Creating Derived Features - Polynomial Features**
  * **Cross Validation vs. Train/Test Split**
  * **Cross Validation for Parameter Tuning**
  * **Searching over Parameters using GridSearchCV**

We are going to make a big step today in the sophistication and kinds of models we will be capable of building. We will talk about several common feature transformations that you can/should perform when preparing most data for machine learning models.

We're also going to learn how to improve the evaluation of the performance of any supervised machine learning models we build using a much better (more accurate) method than train-test split, one that gives more reliable estimates of error on unseen data.

#### Outcomes
By the end of this lesson you will be able to:
- articulate why feature scaling is important, and be able to do it using sklearn
- use a variety of transformation methods to reduce non-normality in features
- handle categorical features when building a machine learning model
- handle null/missing values when building a machine learning model
- use cross validation to accurately estimate model performance
- articulate the strengths and weaknesses of basic cross validation
- use cross validation to choose optimal model parameters by searching across many models simultaneously

### Feature Engineering

When we talk about feature engineering, we are really talking about two things:
* transforming the values at our disposal into a representation that a given machine learning algorithm can use and understand
* creating new, derived features from the available features (using domain expertise) and use them when training a model
* dealing with missing values, as most ML algorithms can't handle unknown values. They must be filled in.

We will begin with the first case, transformation, and talk about the three most common transformations necessary to get data into a state where it can be used to train most ML models:
* **feature scaling**: making it so that all columns (features) range over the same values is very helpful for many (but not all) machine learning models. In some cases, scaling along samples is useful, as well.
* **turning categorical values into numerical values**: machine learning algorithms only understand numbers, not categories.
* **handling missing values**: Models break when you give them `NaN`s, what are some strategies to replace `NaN`s with "good" (i.e. useful) numbers?

Let's import the basics (those things we've seen in the past) like always, and generate some fake data:

In [ ]:
#data handling, model creation/evaluation
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy import stats

#make it so that we only show first 4 decimals for floats
np.set_printoptions(precision=4,suppress=True)

# visualization
%matplotlib inline
import seaborn as sns

In [ ]:
# fake data
train = pd.DataFrame({'id':[0,1,2], 'length':[0.9,0.3,0.6], 'mass':[0.1,0.2,0.8], 'rings':[40,50,60]})
test = pd.DataFrame({'length':[0.59], 'mass':[0.79], 'rings':[54.9]})

### Feature scaling

Let's try to predict the `id` of each sample using the `length`, `mass`, and `rings` columns without doing any feature scaling:

In [ ]:
# define X and y
feature_cols = ['length', 'mass', 'rings']
X = train[feature_cols].astype(float)
y = train.id

To demonstrate this, we are going to be using a different classifier from the one's we've learned about so far. 

This algorithm is called [K-nearest-neighbors](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) and simply selects either the category (classification) or average of the K closest samples to it (computed using Euclidean distance).

We are going to use K-nearest-neighbors where k=1 (the single nearest neighbor, so it will simply assume the output of that sample):

In [ ]:
# KNN with k=1
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X, y)

print("Training data:\n",train,"\n")
print("Single test sample:\n",test)

**what id "should" the classifier predict?**

Hopefully, you said id 2.

In [ ]:
print("Prediction: ",knn.predict(test))

The classifier predicts id `1` instead of id `2` because the portion of the distance in the `rings` column **dominates** all of the other distances (it varies over ~10 units, whereas both of the other features vary over ~1 unit).

How to get around this?

**You scale all of the features so that they all range over the same values.**

There are lots of ways to do this, we will talk about/use **Standard Scaling**. 

We simply compute the mean and standard deviation of each column, and for every value, subtract the mean of that column from the value, and divide by the standard deviation:
$$zscore(x_i)=\frac {x_i-\mu}{\sigma}$$

Let's do that using `sklearn`'s `StandardScaler`:

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() #create a scaler object
scaler.fit(X) #fit the scaler
X_scaled = scaler.transform(X) #transform the data with it

# compare original to standardized
print("original values:\n",X.values,"\n")
print("scaled values:\n",X_scaled,"\n")

# figure out how the standardization worked
print("Mean of each column:\n",scaler.mean_,"\n")
print("standard deviation of each column:\n",scaler.scale_,"\n")
print("Z-scoring the values by hand to make sure we arent crazy:\n",(X.values - scaler.mean_) / scaler.scale_)
print("Final Means of scaled data, per column:\n",X_scaled.mean(axis=0))
print("Final SD's of scaled data, per column:\n",X_scaled.std(axis=0))

You can also scale the data to an arbitrary range of values (instead of scaling them to have 0 mean and unit variance), we specify the minimum and maximum values that the column can take on, using `MinMaxScaler`. By default, `MinMaxScaler` scales the data to the range (0,1):

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmax= MinMaxScaler()

minmax.fit(X) #fit the scaler
X_scaled_minmax = minmax.transform(X) #transform the data with it

# compare original to standardized
print("original values:\n",X.values,"\n")
print("scaled values:\n",X_scaled_minmax,"\n")

print("min and max of scaled values:\n",X_scaled_minmax.min(),"\n",X_scaled_minmax.max())
print("Mean of min/max scaled columns:\n", X_scaled_minmax.mean(axis=0))
print("Std of min/max scaled columns:\n", X_scaled_minmax.std(axis=0))

Other scaling approaches may be more appropriate in different circumstances, you can read about them here: [preprocessing and scaling data in scikit-learn](http://scikit-learn.org/stable/modules/preprocessing.html)

One thing that is very important to keep in mind when scaling: 

**When you're creating a scaling object, you should first "fit" it to the training data, then transform both the training and testing data using the "fit" scaler. If you try to fit the training and testing data separately, you will get inaccurate results.**

(Why?)

#### Exercise Time!!!

* load in the wine data located [here](http://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data). The columns for the data are as follows:
    1. Label
    2. Alcohol
    3. Malic acid
    4. Ash
    5. Alcalinity of ash  
    6. Magnesium
	7. Total phenols
    8. Flavonoids
    9. Nonflavanoid phenols
    10. Proanthocyanins
    11. Color intensity
    12. Hue
    13. OD280/OD315 of diluted wines
    14. Proline
    
* build a knn model with 3 nearest neighbors to predict the wine's `label` from the remaining columns. Don't scale the data, and check the test error when using 80/20 train/test split. (use `KNeighborsClassifier(n_neighbors=3)`)
* now, build the same model, with the same train/test split, but scale the data using `StandardScaler` and `MinMaxScaler`. What is the test error for each of these scaled datasets (for `train_test_split use random_state=1234` for reproducibility)? 

In [ ]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'
wine_column_names = ["label","alcohol","malic","ash","ash_alcalinity","magnesium","total_phenols",
                "flavonoids","nonflavonoids","proanth","color","hue","dilute","proline"]
wine_features = wine_column_names[1:]
wine_target = wine_column_names[0]
wine_data = pd.read_csv(url,names=wine_column_names)
wine_data.head()

In [ ]:
pass

### Feature Transformation: Minimizing Skew in Numeric Features

Many machine learning methods rely on the assumption that feature values are distributed normally and have a symmetrical shape. However, this is not usually (almost never) the case. 

We talked about skewness very briefly when we zoomed through the statistics intro at the beginning of the course, but now we will revisit the topic.

The [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) is un-skewed. An un-skewed distribution is symmetric, which means that the probability of drawing a value to the right of the mean (larger than the mean) is as likely as the probability of drawing a value to the left of the mean (less than the mean). 

We can calculate the skew of a given collection of data as follows:

$Skew = \frac {m^3}{\sigma^3}$
where
$m^3 = \frac {\sum_{i=1}^n (X_i -\mu_x)^3} {n} $


Let's take a look at the histograms of all of the features:

In [ ]:
sns.pairplot(wine_data[wine_features]);

Now lets take a look at one of the features in `wine_data`, called `malic`, which measures the malic acid content of a given wine, and then calculate its skew using `scipy.stats.skew`:

In [ ]:
sns.mpl.pyplot.hist(wine_data.malic)
sns.mpl.pyplot.xlabel("Malic Acid Content")
sns.mpl.pyplot.ylabel("Frequency");

In [ ]:
malic_skew = stats.skew(wine_data.malic)
print("Skew in original data: ",malic_skew)

This is what is called "right-skewed data" (because the skew value is positive). We can try to eliminate/reduce skew using a variety of transformations on the data. We will talk about 2 basic transformation types, and one typical method that "figures out" the best transformation to use:

1. **Square root transformation:** take the square root of each value
2. **Logarithmic transformation:** take the natural logarithm of each value
3. **Box-Cox transformation:** Use the Box-Cox calculation to "figure out" the optimal power (exponent) to transform your data

The [Box-Cox transformation](https://en.wikipedia.org/wiki/Power_transform), **also known as the Power Transformation**, is a very common way to "find" the best way to transform your data automatically. You can think of it as computing an optimal power (exponents and logarithms are actually power transformations) that optimally "normalizes" the data.

Heres an example:

![Box-Cox](./images/boxcox_example_transform.jpg)

Let's do each transformation in turn, compute the new skew, and visualize the newly-transformed data.

**Note:** All of these transformations require that your data is positive to begin with. If you have negative values in your data, you must scale it to be all positive values first (using MinMax Scaler, for example).

In [ ]:
#square root, natural logarithmic, and box-cox transformations
sqrt_malic = np.sqrt(wine_data.malic)
log_malic = np.log(wine_data.malic)
boxcox_malic,power_val = stats.boxcox(wine_data.malic)

Note that the box-cox transformation actually returns both the power to which it raised the data, as well as the transformed data itself (why the output of `stats.boxcox` is saved in 2 different arguments:

In [ ]:
print("Found lambda: ",power_val)
print("Box-Cox transformed data:\n",boxcox_malic)

In [ ]:
#create a big figure
figure = sns.mpl.pyplot.figure(figsize=(12,8))
#add 4 subplots one at a time
figure.add_subplot(141)
sns.distplot(wine_data.malic)
sns.mpl.pyplot.xlabel("Original Malic Acid")
sns.mpl.pyplot.ylabel("Normalized Frequency")
figure.add_subplot(142)
sns.distplot(sqrt_malic)
sns.mpl.pyplot.xlabel("Square Root, Malic Acid")
figure.add_subplot(143)
sns.distplot(log_malic)
sns.mpl.pyplot.xlabel("NatLog, Malic Acid")
figure.add_subplot(144)
sns.distplot(boxcox_malic)
sns.mpl.pyplot.xlabel("BoxCox, Malic Acid");

Now let's compute the overall skew of each dataset:

In [ ]:
all_skews = map(stats.skew,[wine_data.malic,sqrt_malic,log_malic,boxcox_malic])
list(zip(["original data skew","square_root skew","natural log skew","box-cox skew"],all_skews))

So, after using box-cox, we reduced the skew very very significantly (to very close to 0). However, using all of the different transformations **always reduced the skew.**

This does not mean you should always transform all of your features, as transformations like this, because they are non-linear, will have a signficant impact on correlations between your variables, as well as on 

#### Exercise Time!!!
* Look at the individual variable histograms in the wine dataset using `sns.distplot` and compute each feature's original skew (not the targets) using `stats.skew` (a value very far from 0, to either side, is very bad). Which variables are candidates for one of these transformations?
* Transform those columns you think warrant a transformation.
* Compare the test-set error of a Logistic Regression Classifier (`LogisticRegression`) before/after applying your transformations on the dataset (use the `random_state=123` in `train_test_split` to compare identical splits of the data across both transformations).

In [ ]:
pass

### Handling Categorical Features

scikit-learn expects all features to be numeric. So how do we include categorical columns (features) in our model? Before we decide how to encode categorical features, we need to understand the kind of categorical features we have:

- **Ordered categories:** transform them to sensible numeric values (example: small=1, medium=2, large=3)
- **Unordered categories:** use dummy encoding

We will look at one case of unordered categorical columns here:
- **unordered categories with more than 2 levels**

Let's work with another dataset from the UCI ML repository, the [abalone dataset](http://archive.ics.uci.edu/ml/datasets/Abalone), which contains 9 columns:
1. **Sex:** M, F, and I (infant)
2. **Length:** Longest shell measurement
3. **Diameter:** Perpendicular to length
4. **Height:** Height with with meat in shell
5. **Whole weight:** Whole abalone weight
6. **Shucked weight:** Weight of meat only
7. **Viscera weight:** Gut weight (after bleeding)
8. **Shell weight:** Weight after being dried
9. **Rings:** This value +1.5 gives the abalone's age in years

In [ ]:
columns = ["sex","length","diam","height","whole","shucked","viscera","shell","age"]
abalone_data = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data",names=columns)
abalone_data.head()

How do we include an unordered categorical feature with more than two levels, like **sex**? We can't simply encode it as F=1, I=2, M=3, because that would imply an **ordered relationship** in which I is somehow "double" F and M is somehow "triple" F.

Instead, we create **additional dummy variables** and append them to the original dataset:

In [ ]:
sex_dummies = pd.get_dummies(abalone_data.sex).astype(int)
sex_dummies.head()

However, we actually only need **two dummy variables, not three**. Why? Because two dummies capture all of the "information" about the **sex** feature, and implicitly defines M as the **baseline level** (0 for both F and I implies M):

In [ ]:
sex_dummies = sex_dummies[["F","I"]]
sex_dummies.head()

Here is how we interpret the encoding:

- F is encoded as F=1 and I=0
- I is encoded as F=0 and I=1
- M is encoded as F=0 and I=0

If this is confusing, think about why we only needed one dummy variable for a hypothetical binary categorical variable, not two dummy variables. In general, if you have a categorical feature with **k levels** (k distinct values that it takes on), you create **k-1 dummy variables**.

Let's reappend these features to the original abalone dataset and drop the original `sex` column:

In [ ]:
abalone_data = pd.concat([abalone_data,sex_dummies],axis=1) #remember that concatenating columns means axis=1!
abalone_data.drop("sex",inplace=True,axis=1)
abalone_data.head()

### Handling missing values

scikit-learn models expect that all values are **numeric types** and **hold some meaning**. Thus, missing values are not allowed by scikit-learn (or by most machine learning algorithms).

Let's use an example dataset that contains missing values to explore some strategies for how they can be handled. This dataset is called the [Chronic Kidney Disease Dataset](https://archive.ics.uci.edu/ml/datasets/Chronic_Kidney_Disease) and can be used to create a model that predicts whether someone has CKD (chronic kidney disease) or not.

The columns in the dataset are as follows:

* **age**: subject age, numeric
* **bp**: blood pressure,  numeric
* **sg**: specific gravity, numeric but discrete
* **al**: albumin, numeric but discrete
* **su**: sugar, numeric but discrete
* **rbc**: red blood cells, categorical
* **pc**: pus cell, categorical 
* **pcc**: pus cell clumps, categorical
* **ba**: bacteria, categorical
* **bgr**: blood glucose random, numeric
* **bu**: blood urea, numeric
* **sc**: serum creatinine, numeric 
* **sod**: sodium, numeric 
* **pot**: potassium, numeric 
* **hemo**: hemoglobin, numeric
* **pcv**: packed cell volume, numeric
* **wc**: white blood cell count, numeric
* **rc**: red blood cell count, numeric
* **htn**: hypertension, categorical
* **dm**: diabetes mellitus, categorical 
* **cad**: coronary artery disease, categorical
* **appet**: appetite, categorical
* **pe**: pedal edema, categorical 
* **ane**: anemia, categorical
* **class**: class, categorical

Let's load the data in:

In [ ]:
kidney_columns = ["age","bp","sg","al","su","rbc","pc","pcc","ba","bgr","bu","sc","sod","pot","hemo","pcv","wc","rc","htn","dm","cad","appet","pe","ane","class"]
kidney_data = pd.read_csv("../data/chronic_kidney_disease.csv", header=None,names=kidney_columns)
kidney_data.head()

So it looks like any cell with a missing value is marked with a `?`.

Let's make **pandas** recognize that by specifying the `NaN` value type by passing in `?` into the `na_values` parameter of `read_csv`:

In [ ]:
kidney_data = pd.read_csv("../data/chronic_kidney_disease.csv",header=None,na_values="?",names=kidney_columns)
kidney_data.head()

Now let's look at which cells have `NaN`'s in them by calling `isnull()` on the `kidney_data` `DataFrame`:

In [ ]:
kidney_data.isnull().head()

Let's check the total number of rows missing values per column:

In [ ]:
# check for number of missing values per column
kidney_data.isnull().sum()

We are going to have to handle the categorical and the numeric columns differently:

In [ ]:
kidney_data.dtypes

Let's rearrange the numeric and categorical columns for easier access later, and lets remove the response column (`class`) into a separate `Series`:

In [ ]:
print("column names before rearranging:\n",kidney_columns)
kidney_columns = kidney_columns[:5]+kidney_columns[9:18]+kidney_columns[5:9]+kidney_columns[18:]
print("column names after rearranging:\n",kidney_columns)

In [ ]:
kidney_data = kidney_data[kidney_columns]
target = kidney_data["class"]
print(kidney_data.dtypes)
kidney_data.head()

Ok, now that our data is arranged in a way that's amenable with dealing with the numeric and the categorical columns separately, let's work through the ways in which we can handle null values.

The first thing we can do, is simply throw out any samples that have nulls in any column:

In [ ]:
kidney_data_nonnull = kidney_data.dropna()
print("Fraction of data kept:\n",float(kidney_data_nonnull.shape[0])/kidney_data.shape[0])

So throwing out ~60% of the data is no bueno. What else can we do?

We can **impute** (fill in) the data on a per-column basis. The imputation strategy for categorical columns is usually one of the following:
  1. fill in with the most common categorical value
  2. fill in with a special "missing" category

Let's do each in turn.

Here's how we would do **1.**

We are going to get the most common category per column, and store it in a `Series` first, then we will apply it per-column to those columns:

In [ ]:
#function to return the most frequent value in a pandas Series
def get_most_frequent_value(my_column):
    return my_column.value_counts().index[0]

most_frequent_values_per_column = kidney_data[kidney_columns[14:-1]].apply(get_most_frequent_value,axis=0)
print("Most frequent value in each column:\n",most_frequent_values_per_column)

In [ ]:
categorical_most_frequent = kidney_data[kidney_columns[14:-1]].fillna(most_frequent_values_per_column,axis=0)
categorical_most_frequent.head()

Now let's do **2.** (Which is much easier):

In [ ]:
special_missing_category = kidney_data[kidney_columns[14:-1]].fillna("missing")
special_missing_category.head()

For numerical columns, there are 3 common strategies for filling in missing values:

1. Fill in using the mean
2. Fill in using the median (when many outliers are present)
3. Fill in with some default value (e.g. 0).

Let's do each in turn:

In [ ]:
#1.
print("Lots of missing data here:\n",kidney_data[kidney_columns[:14]].head())
mean_per_column = kidney_data[kidney_columns[:14]].apply(lambda x: x.mean(),axis=0)
print("Mean value per column:\n",mean_per_column)

In [ ]:
#1. cont'd.
numeric_mean_filled = kidney_data[kidney_columns[:14]].fillna(mean_per_column,axis=0)
numeric_mean_filled.head()

In [ ]:
#2.
print(kidney_data[kidney_columns[:14]].head())
median_per_column = kidney_data[kidney_columns[:14]].apply(lambda x: x.median(),axis=0)
print("Median per column:\n",median_per_column)

In [ ]:
#2. cont'd.
numeric_median_filled = kidney_data[kidney_columns[:14]].fillna(median_per_column,axis=0)
numeric_median_filled.head()

In [ ]:
default_value_per_column = kidney_data[kidney_columns[:14]].fillna(0.0)
default_value_per_column.head()

All of these imputation methods will have distinct effects on classification accuracy.

Much fancier methods for imputing values, including [EM (expectation maximization)](https://en.wikipedia.org/wiki/Expectation–maximization_algorithm), [multiple imputation](https://en.wikipedia.org/wiki/Imputation_%28statistics%29), etc. exist. 

These fancy imputation methods attempt to preserve the statistical relationships among the variables, and "fill in" the most likely values, given the values of all of the other columns (these are "distributional" imputation methods, as they attempt to keep the distributions of the columns as similar as possible after imputation to what they were prior to imputation). 

**However, they give really unstable/unreliable results when you have lots of missing data (>10% missing values), so I don't recommend using them when you have a dataset that is missing lots of values.**

Look into these to find out more.

#### Exercise Time!!!

1. Create a complete, filled in dataset of non-missing values using mean imputation per numeric column, most frequent value imputation for the categorical values, convert all of the categorical columns into numerical columns using `get_dummies`
2. Do the same thing using median imputation for each numeric column.
3. Compare test set errors when building a Logistic Regression model (`LogisticRegression()`) on the data and using train/test split (`train_test_split(random_state=123)`), predicting the `class` column. Which imputation method seems to perform better on this dataset?

In [ ]:
pass

### Generating higher-order features: Polynomial Transformation

Although some machine learning methods, like **Random Forests** implicitly model non-linear interactions between the features in your data, it can be helpful to generate the full complement of non-linear interaction terms quickly. A simple and common method to use is polynomial features, which can generate features’ higher-order and interaction terms.

Let's take a look at the kidney data, so we can see how this can be done in practice using the preprocessing function `PolynomialFeatures`. Let's first fill in all of the nulls, then go forward with the exploration:

In [ ]:
kidney_data_numeric_columns = kidney_data.columns.tolist()[:14]
kidney_data_categorical_columns = kidney_data.columns.tolist()[14:-1]
kidney_data_filled = kidney_data.copy()
kidney_data_filled[kidney_data_numeric_columns] = numeric_median_filled
kidney_data_filled[kidney_data_categorical_columns] = categorical_most_frequent
kidney_data_filled.isnull().sum()

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly_fit_3 = PolynomialFeatures(degree=3)
fitted_degree3_numeric_kidneys = poly_fit_3.fit_transform(kidney_data_filled[kidney_data_numeric_columns])

Let's take a look at the first few rows in this data:

In [ ]:
fitted_degree3_numeric_kidneys[:5]

This transformation generates every possible pairwise combination of all of the numeric columns in the kidney dataset upto a degree of 3.

So, if we had 3 columns, $X,Y,Z$, this transformation would generate:
$$(1,X,Y,Z,XY,XZ,YZ,X^2,Y^2,Z^2,X^2Y,X^2Z,...,XYZ,X^3,Y^3,Z^3) $$

However, this is complete overkill. What we usually want to generate is just the interactions ($XY, YZ$,etc. terms) not all of the polynomial degrees. In that case, we simply pass an extra Boolean parameter to the `PolynomialFeatures` function:

In [ ]:
poly_fit_3_interact = PolynomialFeatures(degree=3,interaction_only=True)
#don't want the initial constant term, so only keep columns 1 to the end
interactions_only_kidney_columns = poly_fit_3_interact.fit_transform(kidney_data_filled[kidney_data_numeric_columns])[:,1:]

#### Exercise Time!
1. Generate the interactions-only degree-2 polynomial-fit features for the kidney_data null-filled dataset on the numeric columns only.
2. Use train_test_split to predict the class with/without these polynomial features using `LogisticRegression()`. What happens to train/test accuracy?

In [ ]:
pass

Now, we are going to talk about better ways to measure model performance than train/test split.

### Cross-validation as many train/test splits

We are going to motivate this discussion of cross-validation by trying to see if there are any drawbacks from just using train/test split. Specifically:
  - What is the drawback of using the **train/test split** procedure for model evaluation?
  - How does **K-fold cross-validation** overcome this limitation?

**Motivation:** We need a way to choose between several different machine learning models (or multiple versions of the same model). Our goal is **to estimate likely performance of a model on out-of-sample data**.

**Initial approach:** Train/test split
- Split the dataset into two pieces, so that the model can be trained and tested on **different data**.
- **Testing accuracy** is a better estimate than training accuracy of out-of-sample performance (because testing on the same data that you used to train the model causes **overfitting** and worse out-of-sample performance).
- However, just using one train/test split provides a **high variance** estimate since changing which observations happen to be in the testing set can significantly change testing accuracy.

What would be a better approach?

**Better approach:** Create a bunch of train/test splits, calculate the testing accuracy for each, and average the results together.

**This is the essense of cross-validation.**

### Cross-validation flow
1. Split the dataset into K **equal** partitions (or "folds").
2. Use fold 1 as the **testing set** and the union of the other folds as the **training set**.
3. Calculate **testing accuracy**.
4. Repeat steps 2 and 3 K times, using a **different fold** as the testing set each time.
5. Use the **average testing accuracy** as the estimate of out-of-sample accuracy.

**Here's an example diagram where K = 5 (5 fold cross-validation):**
![5-fold cross-validation](./images/5-fold-cv.png)

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

In [ ]:
# simulate splitting a dataset of 30 observations into 5 folds
kf = KFold(n_splits=5, shuffle=False)

# print the contents of each training and testing set
print('{} {:^74} {}'.format('Iteration', 'Training set observations', 'Testing set observations'))
for iteration, data in enumerate(kf.split(range(30)), start=1):
    print('{:^9} {} {:^25}'.format(iteration, str(data[0]), str(data[1])))

Let's go through this carefully:
- Dataset contains **30 observations** (numbered 0 through 29)
- 5-fold cross-validation, thus it runs for **5 iterations**
- For each iteration, every observation is either in the training set or the testing set, **but not both**
- Every observation is in the testing set **exactly once**

Let's try this with real data and compare to just a single train/test split (we will use the vertebral column data we've used before):

In [ ]:
# read in the vertebral column data
vertebral_data = pd.read_csv("../data/vertebral_column_2_categories.dat", sep=" ",
                             names=["pelvic_incidence","pelvic_tilt","lumbar_lordosis_angle","sacral_slope","pelvic_radius","spondy_grade","outcome"])
#generate features, targets
X = vertebral_data.iloc[:,:-1]
y = vertebral_data.outcome
rf = RandomForestClassifier(n_estimators=50) #random forest with 50 trees

#train/test split with 10% of data in each split:
for i in range(10):
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.1)
    rf_fit = rf.fit(X_train,y_train)
    print("Accuracy for split ",i,": ",metrics.accuracy_score(y_test,rf_fit.predict(X_test)))

#compute cross-validation score accuracy across 10 folds
cross_val_scores = cross_val_score(rf,X,y,cv=10)
print("10-fold accuracies:\n",cross_val_scores)
print("Mean cv-accuracy:",np.mean(cross_val_scores))
print("Std of cv-accuracy:",np.std(cross_val_scores))

#### Exercise Time!!!

* Compute cross-validated mean/std of accuracies when doing 20, 40, 50-fold cross-validation using `cross_val_score()`. What happens to the mean and standard deviation of the accuracies as you increase the number of folds?

In [ ]:
pass

Here are the advantages of **cross-validation:**
- More accurate estimate of out-of-sample accuracy
- More "efficient" use of data (every observation is used for both training and testing)

However, there are some advantages to using **train/test split:**
- Runs K times faster than K-fold cross-validation
- Simpler to examine the detailed results of the testing process

We can improve this basic cross-validation approach a bit more as follows:

**Repeated cross-validation**

- Repeat cross-validation multiple times (with **different random splits** of the data) and average the results
- More reliable estimate of out-of-sample performance by **reducing the variance** associated with a single trial of cross-validation

**Creating a hold-out set**

- "Hold out" a portion of the data **before** beginning the model building process
- Locate the best model using cross-validation on the remaining data, and test it **using the hold-out set**
- More reliable estimate of out-of-sample performance since hold-out set is **truly out-of-sample**

**Feature engineering and selection within cross-validation iterations**

- Normally, feature engineering and selection occurs **before** cross-validation
- Instead, you can perform all feature engineering and selection **within each cross-validation iteration**
- This gives us a more reliable estimate of out-of-sample performance since it **better mimics** the application of the model to out-of-sample data

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# use 10-fold cross-validation to find best max_depth per-tree out of depth 2 or 3

# try max_depth=2
rf_2 = RandomForestClassifier(max_depth=2,
                              random_state=1,
                              n_estimators=10)
print("Cross-validated mean accuracy for depth 2:",
      cross_val_score(rf_2,
                      X,
                      y,
                      cv=10,
                      scoring='accuracy').mean())

# try max_depth=3
rf_3 = RandomForestClassifier(max_depth=3,
                              random_state=1,
                              n_estimators=10)
print("Cross-validated mean accuracy for depth 3:",
      cross_val_score(rf_3,
                      X,
                      y,
                      cv=10,
                      scoring='accuracy').mean())

In [ ]:
# use GridSearchCV to automate the search across depths 1-10
rf_grid = RandomForestClassifier(n_estimators=50,
                                 random_state=1,
                                 n_jobs=-1) #50 trees
max_depth_range = range(1, 11)
param_grid = dict(max_depth=max_depth_range)
print(param_grid)

grid = GridSearchCV(rf_grid,
                    param_grid,
                    cv=10,
                    scoring='accuracy')
grid.fit(X, y)

# examine the results of the grid search
grid.cv_results_

In [ ]:
grid_mean_scores = grid.cv_results_["mean_test_score"]

In [ ]:
# plot the results
sns.mpl.pyplot.plot(max_depth_range, grid_mean_scores)
sns.mpl.pyplot.xlabel('max_depth')
sns.mpl.pyplot.ylabel('Cross-Validated Mean Test Set Accuracy');

In [ ]:
# what was best model?
print("Best score:",grid.best_score_)
print("Best params:",grid.best_params_)
print("Best estimator:",grid.best_estimator_)

In [ ]:
# search multiple parameters simultaneously - this will take a while
max_depth_range = range(1, 11)
leaf_range = range(1, 11)
param_grid2 = dict(max_depth=max_depth_range,
                   min_samples_leaf=leaf_range)
grid2 = GridSearchCV(rf_grid,
                     param_grid2,
                     cv=10,
                     scoring='accuracy',
                     n_jobs=-1)
grid2.fit(X, y)

print(grid2.cv_results_)

In [ ]:
print(grid2.best_score_)
print(grid2.best_params_)

#### Exercise Time!!!

I want you to use and test several different imputation methods, different feature transformations, and different model parameters using `GridSearchCV` for the kidney dataset.

This will get you to use everything we've learned today on one dataset with several kinds of missing values, and using both categorical and numeric values.

**Don't forget to transform the categories using `pd.get_dummies()` once you've filled in the missing data!**

Remember there are other parameters you can tune for random forests than just the maximum depth. Look at the random forest lesson or the [random forest documentation](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).

Try to be systematic in your exploration. Your goal is to make as robust a model as you can (lowest average cross-validated test error).

In [ ]:
pass